### Environment Preparation 

* prepare session and bucket 
* prepare necessary parameters 

In [1]:
import sagemaker 
import boto3 
sess = sagemaker.Session()
BUCKET = sess.default_bucket()

In [2]:
BUCKET

'sagemaker-us-west-2-518460648208'

In [3]:
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]


In [4]:
!ls 


01-environment.ipynb			      deploy_model.py
02-a2i-object-detection-and-retraining.ipynb  images
03-prepare-lambda-functions.ipynb	      invoke_image_object_detection
api.sh					      process_a2i_results
augmented.manifest			      README.md
content_image				      sample-a2i-images


### Setup lambda functions and SQS 
We use a lambda function ```invoke_image_object_detection``` to 
1. trigger the sagemaker endpoint to do inference
2. save the image to s3 and trigger a2i process 
3. send a2i and s3 information to SQS 

And, we use a lambda function ```process_a2i_results``` to 
1. listening to SQS and process a2i results 
2. trigger retraining pipelines 

The following steps creates lambda functions 

* copying codes to s3 
* create lambda functions by codes on s3 
* setup environment variables 


In [5]:
%%bash -s "$BUCKET" 
cd invoke_image_object_detection 
zip -r invoke_image_object_detection.zip  .
aws s3 cp invoke_image_object_detection.zip s3://$1/lambda/
cd ../process_a2i_results
zip -r process_a2i_results.zip  .
aws s3 cp process_a2i_results.zip s3://$1/lambda/

updating: lambda_function.py (deflated 57%)
updating: pass_tasks2sqs.py (deflated 45%)
upload: ./invoke_image_object_detection.zip to s3://sagemaker-us-west-2-518460648208/lambda/invoke_image_object_detection.zip
updating: prepare_data.py (deflated 67%)
updating: model.py (deflated 47%)
updating: lambda_function.py (deflated 62%)
upload: ./process_a2i_results.zip to s3://sagemaker-us-west-2-518460648208/lambda/process_a2i_results.zip


In [6]:
!aws lambda create-function --function-name invoke_image_object_detection --zip-file fileb:///home/ec2-user/SageMaker/mlops_incremental_learning/invoke_image_object_detection/invoke_image_object_detection.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role arn:aws:iam::$account_id:role/TeamRole


{
    "FunctionName": "invoke_image_object_detection",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:invoke_image_object_detection",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 1578,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T21:36:49.159+0000",
    "CodeSha256": "6Hva1SZ2Y7AJoapnduIdrEGVvW0BHxLwtyi29aJEo0c=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "21d08ca8-5c1f-474e-989e-76c02ccc745c",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


#### Configure lambda function - invoke_image_object_detection 
* you can also do it by command line - 
```
aws lambda update-function-configuration --function-name invoke_image_object_detection \
    --environment "Variables={BUCKET=my-bucket,KEY=file.txt}"
```    
![configure environment variable](./content_image/setup_env_vars_for_lambda2.png)

In [7]:
%store -r flowDefinitionArn
%store -r endpoint_name

In [8]:
sqs = boto3.resource('sqs')
queue = sqs.create_queue(QueueName='a2itasks', Attributes={'DelaySeconds': '0'})
print(queue.url)



https://us-west-2.queue.amazonaws.com/518460648208/a2itasks


In [9]:
queue

sqs.Queue(url='https://us-west-2.queue.amazonaws.com/518460648208/a2itasks')

In [10]:
bucket_key = "a2i-demo"
variables = f"A2IFLOW_DEF={flowDefinitionArn},BUCKET={BUCKET},ENDPOINT_NAME={endpoint_name}, SQS_URL={queue.url}, KEY={bucket_key}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name invoke_image_object_detection --environment "$env"


{
    "FunctionName": "invoke_image_object_detection",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:invoke_image_object_detection",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 1578,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T21:36:55.334+0000",
    "CodeSha256": "6Hva1SZ2Y7AJoapnduIdrEGVvW0BHxLwtyi29aJEo0c=",
    "Version": "$LATEST",
    "Environment": {
        "Variables": {
            "ENDPOINT_NAME": "DEMO-object-detection-augmented-ai-2021-05-06-21-17-02",
            "A2IFLOW_DEF": "arn:aws:sagemaker:us-west-2:518460648208:flow-definition/fd-sagemaker-object-detection-demo-2021-05-06-21-25-37",
            "BUCKET": "sagemaker-us-west-2-518460648208",
            "KEY": "a2i-demo",
            "SQS_URL": "https://us-west-2.queue.amazonaws.com/518460648208/a2itasks"
        }
    },
    "TracingConfig":

In [11]:
!aws lambda create-function --function-name process_a2i_results --zip-file fileb:///home/ec2-user/SageMaker/mlops_incremental_learning/process_a2i_results/process_a2i_results.zip  --handler lambda_function.lambda_handler --runtime python3.7 --role arn:aws:iam::$account_id:role/TeamRole

{
    "FunctionName": "process_a2i_results",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:process_a2i_results",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 2521,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T21:37:22.671+0000",
    "CodeSha256": "2/V4xfRZf4goMRWpsarY+KNjAPx/4DhjVWM61wtPNGE=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "d9e4c4be-7179-4bec-9465-6edbfb24b43a",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


#### Configure lambda function - process_a2i_results
![configure environment variable](./content_image/setup_env_vars_for_lambda3.png)

In [12]:
%store -r model_package_group_name 
%store -r pipeline_name

In [13]:
bucket_key = "a2i-demo"
variables = f"MODEL_GROUP={model_package_group_name},BUCKET={BUCKET},PIPELINE={pipeline_name}"
env = "Variables={"+variables+"}"

!aws lambda update-function-configuration --function-name process_a2i_results --environment "$env"

{
    "FunctionName": "process_a2i_results",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:process_a2i_results",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::518460648208:role/TeamRole",
    "Handler": "lambda_function.lambda_handler",
    "CodeSize": 2521,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2021-05-06T21:49:21.411+0000",
    "CodeSha256": "2/V4xfRZf4goMRWpsarY+KNjAPx/4DhjVWM61wtPNGE=",
    "Version": "$LATEST",
    "Environment": {
        "Variables": {
            "BUCKET": "sagemaker-us-west-2-518460648208",
            "PIPELINE": "ImageObjectDetection",
            "MODEL_GROUP": "ObjectDetectionGroupModel"
        }
    },
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "83464631-8c77-4ee3-9f42-1d187f8836a2",
    "State": "Active",
    "LastUpdateStatus": "Successful",
    "PackageType": "Zip"
}


### Set up SQS 

* we use SQS to pass the information between lambda functions in this solution 
* set up SQS and attach it as a trigger to a lambda function 



In [16]:
!aws lambda create-event-source-mapping --function-name process_a2i_results  --batch-size 10 \
--maximum-batching-window-in-seconds 300 --event-source-arn arn:aws:sqs:us-west-2:$account_id:a2itasks


{
    "UUID": "5d64d012-964c-478c-85ff-872e8f2c5cd8",
    "BatchSize": 10,
    "MaximumBatchingWindowInSeconds": 300,
    "EventSourceArn": "arn:aws:sqs:us-west-2:518460648208:a2itasks",
    "FunctionArn": "arn:aws:lambda:us-west-2:518460648208:function:process_a2i_results",
    "LastModified": 1620337973.653,
    "State": "Creating",
    "StateTransitionReason": "USER_INITIATED"
}


### Create an A2I event Rule
* create a role `SageMakerExecutionRole` and have it trusts cloudwatch events 
* create A2I event rule 
* edit rule, have it targets to the SQS queue we created 

```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "",
      "Effect": "Allow",
      "Principal": {
        "Service": "events.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}```

In [ ]:
!aws events put-rule --name "A2IHumanLoopStatusChanges" \
    --event-pattern "{\"source\":[\"aws.sagemaker\"],\"detail-type\":[\"SageMaker A2I HumanLoop Status Change\"]}" \
    --role-arn "<your execution role>" \
    --region "us-west-2"

### Test Material 
```
{
  "content": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBYWFRgWFRUZGRgYGBgYGBoYGBoYGBgYGhgZGRgYGBgcIS4lHB4rIRgYJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHxISHjQrJCQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIAMMBAgMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAEAAIDBQYBB//EADkQAAEDAgQEBAMGBgIDAAAAAAEAAhEDIQQSMUEFBlFhEyJxgTKhsUKRwdHh8BQVUmJygiPxB5Ky/8QAGQEAAwEBAQAAAAAAAAAAAAAAAAECAwQF/8QAJBEAAgICAwACAgMBAAAAAAAAAAECESExAxJBE1EiMgRhgXH/2gAMAwEAAhEDEQA/AABjTG6aKyAZWC67EhcDbDsWLn2QtWoVC3EAqUEKbYdgd9KdUFX4c0o6u6NEmGVomIqTw0dENW4SDstA4BDuTUmBS0uDiVLU4SNCrunAXKjgm5sADDcHaRGUKj4hwlzXWG61rKkBQYhwKcZtOxUYd+FcNk0UndFr3YVpSfgGwtPlY8mNLT0SAK1zeHN6JzeGN6I+VBZksvUKMha+rwodFEzgwnRC5UKyhZmy2BQtQkm62/8ALQGqlxXB5d5QlHkVgmZ9JXL+COCHfwp4WqnFjtFckjv5a9NdgHBO0FoDSUpw7ui4aThsU7QyNJSCiei4WEbItAMBSKSUIASSRCSAEkkkgDXGuVzxVASmBy5EhBza6kOKIQTXJr3lNxFQd/FSnnGQFVhyeRZLqJhYxqe3EKtIU9JKgDhXSa8ygy6EvGRQFk+quEygGVC4wFe4HglR0F8U2/3fEfRv5otIqKlJ0kQ4PDh13TlHTUnoFd/wdF7LMLSdHAkkHvJgqQiixuUbD7h17pYbiDG7SD1tH5LGUm3g7YcUVGmsgNPgDyfLUZHW/wAwmYzg1anfLnb/AFMkx6iJC0+EOe9o7z+KPZiabCJqNHUG6PkfoPgi9HnLnwuU6q9QxWDoV2Fr8pkGHNABaToQUPgeWMMwAtYXnWXmflp8k1NUYvgdmCbRe4SGOI6wYXKZG4Xrjn5GEaNANsrdIVZR5awrpc5mYuuczjN+gBgeyblHwn4JHmtVzUK9jVveJcq4UuysL6ZO4OZs9CHfmlheWcOxozAVXaOLiW/+rZshTig+CVmDZRaVFWwjdoXoLKGHpkinTDHdYLj9529E19U7NbHVoAjuRCPmXhov4rrLPPsNwd9Qwxhd6Cw9ToEY/k6pEuLG9nOP1AIWsdiHTrPr+Sc7ENcMpspfO/DSP8WK2zB47l+pREvZDTo5pDmH/YWB7FVVTBjovTWPyTo5hEOY64c3cELMce4c2k/yXY8B7JuQD9knqD+C0hy9jDm4emVoyD+HjonNwQ6K2cJ2SFNbdznKl3D5GiFOAutHkEIOsxNTHZWfwASRmUrqfYLDGUJTH0YKJpPSqGVnYwRzEwMRLmKNzYRYEMQkSpMspgYiwIi1PYYTnNUYclQrHuN04XSaJTiIQInw9QsJe0eYAlvr+cSh8XzBVcfjOnX8FzOg8XQnzN9x36hHVXk24pUqHjiT+pU9HiD5EEkuP00CqQYN0XhjLgRtum4pG/Z/ZccR5kfTbkYb6SCbnc+nRZ3+ZVnuBL3EylisI74j6W2RXC8KHvAPwi59AqqEY3RPaUpbwekcv8ScWMzAxb1J6wt3hXktDoi2i8xw/GKVDLmgWnsBt7q84TzjTqmA6DMRtA3XG00rrB0YeLNgAXlzSYGVxI9GkoWhXd11+SJ5ZqNqB5I1zN9oVFUxDWPex5gA2J0hS1UVL7sE/wAnEvThcw819x++qz/N1Ooyk57ZMAzHyPqoqfMtNj8hqNE6EuhaV9dlRmUwQ6x/RKsZQ7aPH8BzkWuAcTGkm/4LT8O4uysYY4B/TZ3p+SxfNPAhTqPDLQ4/mFV8IbUaZBjKZB6EdCuh8UJR7QZHyyUqkj1l7JvEOF+nuFVcRxTWAuecsW/6R2D4wx1NheQX5fNpqLGR+9VVcb4hScIDG37yfumAsEldGzdK7HYfEB1Jry17swJADg2GCwJsTJIMe3VVfF6pc5l5bkGWdQNCD3kIetxBxblFhA+URHawQufSSt4xrJx83KpKkyVRl9106KGbrQ5iYlNc0LhNlHmQJnMgSTl1FsQxlRPc9KrRhOpM7JlCYDqlUKMY2yGfR1QDQM1yTinChdJ1IpEkVRQEoh7CozQKYDfEXDUXXUCuHDlNIdHJXXFMa0p5TBCYAdQnuaFxgXXBSxj2MEQpOG4bK5xm0W97/gowYRVB3lJlTLRpwvNFdj8F4jrHzDrpCs+W+WanjDzNiL7g9h30+9Op4HP5i8NHpeB6LX8Ir0KNMPfNogadSJ7mFPeTj1R0qMU+zN5ytg8gaAIDRfue6B5q4CXguYGk31EiFQYPnOS4spuIsBDgIJMCVpeX+YxiC6i9jvEa0lwdAcWyBItfULX4bhTM/lqdo8pfyjUc5xcXQTldo2zbi3QbEL0Hlrh7WMaA5xDRAzOLr+6t+NcGNsroB+1Egjo4dUM0CiweabWgRJ9FzzlJvq/DaPWrj6ed89tjEubPlLWu++fnqsy58aLU84DNUncMbJtrLjH3ELLuYtOJLqc/M25UJlSVMAhskKRj1ZiSOXIXQ5LMihCkppUkrhRQUNldyroCdMIoBmQpKXOEk6YqQhVlPY8KLw4TgEgoKZXCe9wIQDmFRlzkUCDMi5lUdOoVIXooZwNvdEeGIQbqhldNYwl1AkewJll2kZ1XKohOgGeCCoX0EQEg2UACikniknudCZnQA0005php/f73XAV10EaxEEe179kpLBUHUkF4WuAWg2uB6g6fijuNYx9TDupC0GZGttB81VcWwrm0i9mkNe0ja6bw/iQqAtLocR5mnQ9x1CcI1+RvOVqjPYZxaS3PAMAwT5TJE+0fNek/+I8C5uIdWNUuY1mT+0ucQXBpJkgR2uvOcVSptcZdfNcC8X6jZej8kcwUKNFzc7Z11yxGgJN102YHp3NXFGUcK+pJMRA1MyNFh38S8anTqmxJyR/cQTMeyp+LcXdxF4oUSW0ZzPfoXwW/1XiJ7kxoNdlyvwSS177MYZYztoCfYBcvNBykkjfjkoxyUXH+DEMNR5Azhoa37XwgG+5Xn+Jplji06g/9Fem88uL8Y0D4W0wBqbkmbfcvPeOUyHj/AB1PqUsRfVCmrj2YCxdKa0JwKujnOTCRK48pjE6AJY+y4EwBdDUBY8PSe6UxwhNzpCsV0kvEC4jsItxQldGHurQUhoFK3DBTZp1KR7OygqM7K6xVMdFBTpgo7B1KtrCphRI1CsG0bpzxKVh1K4YeV11EKwDAntoAosVFO+idkvDO6tBSE3ThRBTsKKhrF1jJVwcM0KIUgEdvA6lX4EqB1FXooKF2HCaYqKplJONIaQrNmGB0RLMFNgJhDd4KSoi4dRz0sr7QS2+mUjT0us3U4Pkqmm6A5zXZO9iT8gfvWqxOJyQ0jaD0lA8QoGs1rmOAqMByE76QAdiri+rLu2ZkcEOkbj63Wm4TwSmKbxiDkDv+Ng3L3tcWH08s+yhwPNLaYIq4QmoCIc13lJ7tIstNw59TFllSrTbSptcHBhIzPI0uR0kbfEffZtGlxStGo5Q5dpYZjAYLy3M+e9hbQDULR1MSxj8jdSJPtsqHhwqOqOc9rhmFyLgCIG+yu6GFp2MS7QTss1oybtlPx3AzUa/KYy3cBN9gs5zByz4tPMweZoJZ/deXMPrt3XqXgiIOioeKUgHQ0RfQad1hzRcfyTNYSUl1Z4ezCHQjRNfhey2vNHDPDf4jR5HmT0Dzcj3uVngJKcZWrMJR6uipOGURoFW9SneEhTA1VWTRU+GUvDKtHsErhYITsKKt7ZUIZdW3gjRdZhQCkKiq/h+xSV/4DUkBQeydVwYqLJ+HqWQ1dt1kW2OqvlDiWlTNlJ7eqdBY9olNewhcBI0Uk9U6GRN1RTBZQ09ZTnPG6SQqGVmrlJymzghQYcyYVD0Odqk9kXUlamBdNAzeil0sieBuddYQpXUQdCh3UyClGSehJ4Jm0byp8HZ+utvyQZqGLIjhzC57fWVcbspBGNwuYEbqh8CPKTBBsfpC1WIYZsEFiKDHxnEEHUReFpJDRDgwx13tBdu4tABP7hX/AA54eHUnR8JyWgAi7dFFgeDtPwmw7D9J2ROH4U5rpDmxmsTMiDe4uNxupbaKikyy4RiX5AxpfrveOy1OEpwJce/Qqnw4ywA6ZvbtrfZWmHcSeoVRFIsWGUPxDCBzZA8wuETSTqxsfRXOKlFpkxbTwYzjODD6b2ET5SW/5C4XnIIb6r07HOgx2K874jhQDmA66dVxccqbRvyxtdgCq64JTKhulVqhOosButbOehGFG6l0UtWn0TaQKOyYtkGUyngKQtukXABJyoUmkR5ikn5uySnuT2D3EBMIBUlOnm12UlKnJ00Qk0WlggZTJcOinLBOiKFMLjqZBkXQUC1RA0QzKbiVYam6Ip5WppCor6dM9FFVwpVg/wCKdihnvgmUk2NkDKDhpom5YMoh2IMABRYhx2CdhSHsEi6ieSNkwVSLFEZ5Gil2weUDsc4qSgSTB0RTQMkkKNjxeE44EkkQVG3MKfhNQh8dj9FD4l0qZh2b1Vp07GlbLx7utwi8Ng21D8J/1k/NC8NcH6wJ6q+/k5DbSJ6GxXRtD0GcNwQZ5XAjYZomehg2Vo2iwGIF/wB7rP4Z+TyukHrNj6zoiGVMzhc/eoWBsKxNZjHEQQNo2JROG4nSFhmn/FR4/DtIzbIdlNpNvdVoRo6FYOEt0TsTUAbqqrDPyugaKTHPdIt+ScpVFhGNsqOKN8xOyyHEsoYT0d9bfitjxIyDbQLG8bHkcB2+oleev2OmX6FFVc03hQvrCLJswLqNlOdFvSOR5HGuUn4qEQWNAAUL6QzRsmkhJEuGqA6rkCUjQACYwQjAMfASTfDHVdRQYLV1AsGZS4Zh1K7XJMAqdlOQob8G/oY8rrRNgoatM6dE6gYN0k3YqZ12G1umuGVhUue5hQvqzbYaqrKqkCUMzutlPVph0HfdE0y0adLqFo887FIWyM0w1wJ0TKwzOFrKxq0xv7KAggTCd+DS8IG4e8kWClZTgF0WRAe0sJNioXPLmgDRCCiBtbMCIsmsokbJ9NwY2SLSp6r9C3dF+j3kHdoLaJuNgttspGVQTA21UmJALHAawUWEcMfy/wCV2ZzrDYR8zqvQsFjmPYBoR7ELFcvYIvYSW7fhCt8Dw17I809Jm/a6h8soukdKgpLJd43CAtzWWcw9fLULdpWjq4kMovzkCAd1g34uX5mrpcrSZjVOjWcW4jDMo1gFZ6hxZwdrH5LtWqXAON7KnxVTKZCbb2SjUt4kYDswgant6rmH44+q8AAwPU/UarJsxudjmt+KQ33NgfqtfwSm1rAGiwGvU3ue65+abqkb8UVst8U2Wz1aZWO4405JA+IgBbl7paFhOZJDwxn2Jn3/AEWaWQm/xaM/icNbulQow2US9zgbiU1rXAXFlon9nKo0DvbFkOHkOhWzKQ33TBgW5rXVJjoHLbXTcRTltkY6gu0sOfZFhTsqfAPVJW2XskixUi0xNNoEoZmINxsEXVF9fZcr4fSNwpoojw+YDM4WOiY+mbuUhaWsh5k7BDGsbNE7J2BPQpF11ABE23hWAEXE2GnVNa+ZJHeEMAdrCRA1KjewsIEI6k9okxB2Ujhm9OqT+gKr+Jc4y4aaIwMloKDreVxbsiWYkmGgbITBsixFIWaTGYp4pgGBpomPZL2z9ko6vVAdLR2TDwEdQtBvdReDY9Roj2sIAdsUI6pBch6Ahw9K56zdTGBMCbGfuTsKYc4kahPADWGR5nAj71KQKrL7l+uMhbpb6K8wjg45Tvp6rz7g+LLX/rZbHD1SRLetuxSkns6lnBTc+4ktc2ju5oe70kgfQrLMqWhaXn6jm8LEaOA8CoDsfM9h/wDv5LIStLXhjK/S8p4mGBVON8036rj6ktDeikwGFNR7GAwXECTt1Psm5dsErBY8h8vGs573khgIA7ka+116GOFeHGUS3r27qy4Nw9lGk2mwQAPck3JPeSjy20dlq+FNZ2NcrWjNYp+VpHRYvGed7ndSTKv+N4rzvpg6Egn20+aochgCNd1yqLtj5HdIFpYa8qDEsdtBurxtIBgEKsbTJcRBsrZD0QmkIEld8rW21KJGHBaUIyh5ydgEqFsTaTnRGv4JPYS4ibiy7hn5Sb3UrGfaPcosLAPDd1SRXjDskpsLRZOy5p2At6phY4X13soalBpEOdMaAdUTTpwASU78E8AeOY98ZbRclJjIZLrkHXsj2vF56dIlQCsxxiPKQhO0CyDOxT5GQW6rj6paQALm8o52Roa0aaJ2IcA3L133SavYmrIW1IcCRqNFI6mbAm0yY2GyFrNI8wM2hcw1Qxc/EIgpppDqjmOZNgNxB6qYPDQDG111gmGkjsigWFhYRfcpoa/sDxFAkAt3Ejsm0BaHbI+ocryGXsIH1KHqSLRrc9ymxNHK1VobF5GnSUPSbml3TVKrTJiGz1SwRnMGjVTkYmOBGvmCcxs63Kc+kGuEtk9dkZSoZrsItr3TX0CZS1aJY8dDcHv0K9J5W4eGMOaC4w70svO+MVXCm5wjyOafuN16by2ZoMeT8QmT02W/Ck7sJSdIF504U2rhK8Nl7WF7Y1LmDMBbUkAj3Xj+AJeMoBJGkX/7XvzKrSYDgT6rEcxcuspVRXpeRr3f8jRGUO/qaNpOo6p8kE8oSk9Hn7qBGv6q85RwZfiaZIsCXH2B/RS4pmYg6iTeNI0Wo5CwZyvqOGrsrf8AEakep+ijjjchtmybog+L4o06T3jZpj1hGtCoObHODGR8JfleOxEj5hdL0QYfEvIaHmS57jJ9bk/RPfWOUEjQW/fsiuN0w0sDfhAJM9SqmpUdodtLLhSawzWT7Owo4kua0gqKnWIJFpO/Zcp0BldJhxiFG6mdBroTO6b+xaHY5xbGUz1UNFtySNpKTKLgMsiZTcNVcHunTf0CS/slHXsBEgKei5rmEdFHVuCR8M2UT3Whvb37JWCwxuXskjG0XwPKkn1GOZTJIaPsiSVM5pI8okk23v0TGvJGZrSLkO7jpKcxjpgyA3zAQpik0Rs5iGuiLE7t6dVXV6TgczB0kKw8W0kOBOi7lJhgudZ7I65GmBeLmcGvERcdlNXJJ0kDdS0cK0ODyPNJEnS3ULrKbs+YeYGTA09AihtWyZlItYHGJ1ynYIFkuu34h5vWeiMqsc5jnEXFo0tuhKbHAtY0AWtPT1T14JANZ7/EzQQALd1aNcSJLdQCnNfmIkD23jYItktGVwEuNwNh0lLDHvYJQxOd0htxA/RTPpPnQEjYnRDswuXOWzBNvVcwdUvBaTBbr39U442LQTWeAC82AOUDqYuoWGAMgOZ2oAn9hTupjKM1xJ9j1hLh77Py7yJjZV6CYBXD3ENIOt/RdoPcxttydFY4t4BY0C9iSO43QviDxcjT8IMnp1RVDA6+Fzse12pY6B0sYPrKIxvNQo0WU2vAaKTSASPhDToNzYe0qUsJzECQ5sT0jqsVx7hDzVaARGRzBmmMjiTY9QSR6ALTjlVjLvB/+SqjmEljWFrmhzgZsS6C1ojTyg3KvsJzd/EANcWkOny3u4CWuA2vIhYXhnJpMB7iWnUC3uDC1OD4cykcrGRlEAm5deZLlU31Vh2VFg+sfOyLOEgdFt+UCf4ZhdYmY/xBIb8gF589rg62oMQfxWw5e4kxwYxpBjM0xMBwixP+rrdkcLy7E8mtBVNzU5ow7i7QFpHrmH6qehiTlcbE+YtA94H4LPczcWY+gaYcDUztGQXJElaSlgHGjMYnGF5G4M6bKTxRkDRBMx3ChqUpbPwlsTFtUxtGCHToR7rktpheAhxizm6iL7nZVbKD2u1Nzp0V3iCHDPN7QNRG6GZiQ55tobH6pyBjXUjAgibn23lDeH10Fz3CmL5e63l0/ROd5hMRt6KW1QXaAKeKaXuYJLdk3MWlvc2PvuiDRIJbF5F46p9TDicua4+UpJWGy3ZiLC40C6qvKz+o/ckrt/Y8ljiWwbW1+pQ+GeSDJJnXvdJJJaJZeY6g3yCBAbooOL0gDTIES38l1JOWmUAvYIIjdR4Wzvl8kkllHz/RLQ2qLn0H1T8WwZGmL6T/AKlJJOQIFw7bjtP0CN3aerSuJKVofgqf2RsSbKPFMANhrKSSa2J6HYNsi9/N9FC+zXRbyn6pJKvCfoJpsGVhi5YPqVVlgFV9tXD6JJKXtif7FjR+A93ge3RRcwYZnggxcPsekriSqO0UN4V8KO+27s0JJLfm/Vf9BbG1qLYeYvDbyVk34t9LHUKdNxYx1RxLW2Bza/U/eupJQ2UjR4jidVlSnlqESwk6X/5ag/Afcp6dIZ3GLnKSdz5Ekkp/qUypYJeJ3cQe6Kx9qrgNMuiSSyWjN6Ez4PcKFjfID/e76lJJNAywawZXW3UlKmC15jqfkkkmg8BKHxj0/BQYqg3MDH2ep6pJJS0xouaOGblFth16JJJKCj//2Q=="
}
```


## Configure API Gateway 
* Finally, we need an API to have the service accessible
* This API accepts image POST to it and pass the image to ```invoke_image_object_detection```

```curl -X POST -H 'content-type: image/jpeg' --data-binary @$f $OD_API | jq .``` 

* we can create it by console also by aws cli 



#### Select RESTful API 


![configure environment variable](./content_image/build_api_gateway.png)



### Create Resource and Methods 

![configure environment variable](./content_image/create_resource.png)
<br>
![configure environment variable](./content_image/create_method.png)
<br>
![configure environment variable](./content_image/create_method2.png)
<br>



### Configurations for passing the binary content to backend
* Add binary media type in ```Settings``` of this API 
* Configure which attribute to extract and fill it in event in Lambda integration 

![configure environment variable](./content_image/setting.png)
![configure environment variable](./content_image/integration.png)
![configure environment variable](./content_image/setting1.png)

In [17]:
!cat api.sh 

#!/bin/bash

#OD_API='https://2lvd1jdy8l.execute-api.us-west-2.amazonaws.com/test/classify'
OD_API=$1
echo $OD_API

for f in $(ls images/*.jpg)
do
    curl -X POST -H 'content-type: image/jpeg' --data-binary @$f $OD_API | jq .
done


In [22]:
!./api.sh "https://jwww8ados2.execute-api.us-west-2.amazonaws.com/dev/detect" 

https://jwww8ados2.execute-api.us-west-2.amazonaws.com/dev/detect
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62969  100 26368  100 36601  28413  39440 --:--:-- --:--:-- --:--:-- 67781
"{\"prediction\": [[7.0, 0.2924271523952484, 0.13341665267944336, 0.0, 0.7542880177497864, 0.8702528476715088], [11.0, 0.04573383182287216, 0.04162609577178955, 0.0, 0.8578957915306091, 0.8826754093170166], [4.0, 0.04551791399717331, 0.022185251116752625, 0.3733759820461273, 0.24723084270954132, 1.0], [15.0, 0.03441595658659935, 0.01854536682367325, 0.4023258686065674, 0.24219605326652527, 1.0], [4.0, 0.023586826398968697, 0.1888982355594635, 0.5771069526672363, 0.3089553415775299, 0.9741657972335815], [4.0, 0.023574532940983772, 0.31059515476226807, 0.5873633623123169, 0.38036471605300903, 0.7068634033203125], [4.0, 0.022853676229715347, 0.3659321069717407, 0.7094444036483765, 0.42487066

100  223k  100 20517  100  203k  23078   229k --:--:-- --:--:-- --:--:--  251k
"{\"prediction\": [[11.0, 0.3860365152359009, 0.20666053891181946, 0.13779973983764648, 0.9189637899398804, 1.0], [7.0, 0.3361562192440033, 0.2122941017150879, 0.14502698183059692, 0.9476815462112427, 0.9880970120429993], [14.0, 0.027463899925351143, 0.5604172348976135, 0.2389209270477295, 0.6450697779655457, 0.34024953842163086], [14.0, 0.027399279177188873, 0.5586913824081421, 0.2831738293170929, 0.6433795690536499, 0.38897308707237244], [14.0, 0.026165975257754326, 0.5046780109405518, 0.18500763177871704, 0.5891914367675781, 0.27858689427375793], [14.0, 0.025359081104397774, 0.4658701419830322, 0.1586330533027649, 0.5372577905654907, 0.22945073246955872], [14.0, 0.024278927594423294, 0.46107733249664307, 0.18672394752502441, 0.540502667427063, 0.2844684422016144], [14.0, 0.02400284633040428, 0.36461594700813293, 0.24125559628009796, 0.44018593430519104, 0.33533579111099243], [14.0, 0.02339911460876465, 0.

100 27906  100 21422  100  6484  34946  10577 --:--:-- --:--:-- --:--:-- 45449
"{\"prediction\": [[11.0, 0.36658504605293274, 0.30325841903686523, 0.04849529266357422, 0.8349888324737549, 0.9091528654098511], [14.0, 0.15432827174663544, 0.25969645380973816, 0.0653243362903595, 0.8716251850128174, 0.9088501930236816], [14.0, 0.02965421788394451, 0.762209415435791, 0.12172560393810272, 0.9216351509094238, 0.3811876177787781], [11.0, 0.029477346688508987, 0.3134917616844177, 0.08254463970661163, 0.7568491101264954, 0.5295392274856567], [14.0, 0.0270673930644989, 0.6602473258972168, 0.12317241728305817, 0.826054573059082, 0.37967032194137573], [14.0, 0.02683699131011963, 0.4856424927711487, 0.6721611022949219, 0.6272643208503723, 0.9732494354248047], [4.0, 0.025284050032496452, 0.6793282628059387, 0.7430582046508789, 0.742065966129303, 0.890886664390564], [14.0, 0.025242572650313377, 0.44451773166656494, 0.05768585205078125, 0.6585469245910645, 0.24732229113578796], [14.0, 0.02517662383615